In [123]:
pip install geopandas shapely

Note: you may need to restart the kernel to use updated packages.


In [124]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
import numpy as np

## 대여소별 가장 가까운 대여소 거리 구하기

In [ ]:
# CSV 파일 경로
file_path = "/Users/minjikim/Desktop/sesac/프로젝트 1/contents/seoul_bicycle_masterinfo.csv"

# CSV 읽기
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
print(df.head())  # 상위 5개 행 확인

   대여소_ID                      주소1            주소2         위도          경도
0  ST-999       서울특별시 양천구 목동서로 280  목동아파트 8단지 상가동   0.000000    0.000000
1  ST-998       서울특별시 양천구 목동서로 130  목동아파트 4단지 상가동   0.000000    0.000000
2  ST-997       서울특별시 양천구 목동중앙로 49  목동3단지 시내버스정류장  37.534390  126.869598
3  ST-996   서울특별시 양천구 남부순환로88길5-16     양강중학교앞 교차로  37.524334  126.850548
4  ST-995  서울특별시 양천구 중앙로 153 공중화장실            NaN  37.510597  126.857323


In [126]:
df["대여소_ID"]

0        ST-999
1        ST-998
2        ST-997
3        ST-996
4        ST-995
         ...   
3390    ST-1002
3391    ST-1001
3392    ST-1000
3393     ST-100
3394      ST-10
Name: 대여소_ID, Length: 3395, dtype: object

In [127]:
df['location'] = list(zip(df['경도'], df['위도']))
df['point_location'] = df['location'].apply(lambda x: Point(x))
df['point_location'].head()

0                     POINT (0 0)
1                     POINT (0 0)
2     POINT (126.869598 37.53439)
3    POINT (126.850548 37.524334)
4    POINT (126.857323 37.510597)
Name: point_location, dtype: object

In [ ]:
# 1️⃣ point_location 이미 Point라고 가정
geometry = gpd.GeoSeries(df['point_location'])

# 2️⃣ GeoDataFrame 생성
seoul_stations = gpd.GeoDataFrame(df[['대여소_ID']].copy(), geometry=geometry)

# 3️⃣ 현재 좌표계 지정 (WGS84 위도/경도)
seoul_stations = seoul_stations.set_crs(epsg=4326)

# 4️⃣ 미터 단위 좌표계(Korea TM)로 변환
seoul_stations = seoul_stations.to_crs(epsg=5179)

# 5️⃣ 거리 계산 (벡터화)
dist_matrix = seoul_stations.geometry.apply(lambda x: seoul_stations.distance(x))
np.fill_diagonal(dist_matrix.values, np.nan)
seoul_stations['nearest_station_dist'] = dist_matrix.min(axis=1).round(2)

In [132]:
seoul_stations

,대여소_ID,geometry,nearest_station_dist
0,ST-999,POINT (-5899757.108 17790114.866),0.00
1,ST-998,POINT (-5899757.108 17790114.866),0.00
2,ST-997,POINT (944303.993 1948528.325),218.83
3,ST-996,POINT (942613.203 1947424.115),260.70
4,ST-995,POINT (943201.45 1945895.954),162.59
...,...,...,...
3390,ST-1002,POINT (944914.111 1948026.093),137.72
3391,ST-1001,POINT (-5899757.108 17790114.866),0.00
3392,ST-1000,POINT (944038.679 1945866.202),250.53
3393,ST-100,POINT (962328.201 1948679.677),284.26


## 대여소별 일주일 대여건수 구하기

In [ ]:
file_path = "/Users/minjikim/Desktop/sesac/프로젝트 1/contents/tpss_bcycl_od_statnhm_20251011.csv"

# CSV 읽기
df = pd.read_csv(file_path, encoding='cp949')

# 데이터 확인
df